In [1]:
# !pip install jmetalpy

     |████████████████████████████████| 110 kB 916 kB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 275 kB/s eta 0:00:01
  Created wheel for jmetalpy: filename=jmetalpy-1.5.5-py3-none-any.whl size=196216 sha256=da246acbfe1237b2a09b98b1b6ab67c204e9a4bacbca0b7c4b37079ec760849b
  Stored in directory: /home/siribarne/.cache/pip/wheels/fd/75/33/354a1b9945bad371cdd41bd4b1483e3962129fa81c10583775
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=e71f9b49720aa32c2ce82952224124397eaa5cfe6d59967e62ec1127b0ed1d7b
  Stored in directory: /home/siribarne/.cache/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built jmetalpy retrying


In [13]:
from jmetal.algorithm.multiobjective import NSGAII
from jmetal.operator import SBXCrossover, PolynomialMutation
from jmetal.problem import ZDT1
from jmetal.util.termination_criterion import StoppingByEvaluations

problem = ZDT1()

algorithm = NSGAII(
    problem=problem,
    population_size=100,
    offspring_population_size=100,
    mutation=PolynomialMutation(probability=1.0 / problem.number_of_variables, distribution_index=20),
    crossover=SBXCrossover(probability=1.0, distribution_index=20),
    termination_criterion=StoppingByEvaluations(max_evaluations=25000)
)

algorithm.run()
front = get_non_dominated_solutions(algorithm.get_result())

plot_front = Plot(title='Pareto front approximation', axis_labels=['x', 'y'])
plot_front.plot(front, label='NSGAII-ZDT1', filename='NSGAII-ZDT1', format='png')

In [ ]:
def cross(a, b, i, j):
    return a[:i] + b[j:], b[:j] + a[i:]


def insertion(a, b, i, j):
    # print(a, b, i, j)
    if len(a) == 0:
        return a, b
    while i >= len(a):
        i -= len(a)
    return a[:i] + a[i + 1:], b[:j] + [a[i]] + b[j:]


def swap(a, b, i, j):
    # print(a, b, i, j)
    if i >= len(a) or j >= len(b):
        return a, b
    a, b = a.copy(), b.copy()
    a[i], b[j] = b[j], a[i]
    return a, b

In [31]:
from jmetal.core.problem import FloatProblem, FloatSolution
from math import sqrt

class VRP(FloatProblem):
    """ Problem VRP.
    
    IDK
    
    .. note:: Bi-objective unconstrained problem. The default number of variables is 30.
    .. note:: Continuous problem having a convex Pareto front
    """

    def __init__(self, number_of_variables: int=30):
        """ :param number_of_variables: Number of decision variables of the problem.
        """
        super(VRP, self).__init__()
        self.number_of_variables = number_of_variables
        self.number_of_objectives = 2
        self.number_of_constraints = 0

        self.obj_directions = [self.MINIMIZE, self.MINIMIZE]
        self.obj_labels = ['x', 'y']

        self.lower_bound = self.number_of_variables * [0.0]
        self.upper_bound = self.number_of_variables * [1.0]

    def evaluate(self, solution: FloatSolution) -> FloatSolution:
        g = self.eval_g(solution)
        h = self.eval_h(solution.variables[0], g)

        solution.objectives[0] = solution.variables[0]
        solution.objectives[1] = h * g

        return solution

    def eval_g(self, solution: FloatSolution):
        g = sum(solution.variables) - solution.variables[0]

        constant = 9.0 / (solution.number_of_variables - 1)

        return constant * g + 1.0

    def eval_h(self, f: float, g: float) -> float:
        return 1.0 - sqrt(f / g)

    def get_name(self):
        return 'VRP'

In [32]:
from jmetal.algorithm.multiobjective import MOCell
from jmetal.operator import PolynomialMutation, SBXCrossover
from jmetal.problem import ZDT1
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.util.neighborhood import C9
from jmetal.util.archive import CrowdingDistanceArchive

from jmetal.util.solution import get_non_dominated_solutions
from jmetal.lab.visualization import Plot

problem = VRP()

algorithm = MOCell(
    problem=problem,
    population_size=100,
    neighborhood=C9(3, 3),
    archive=CrowdingDistanceArchive(100),
    mutation=PolynomialMutation(probability=1.0 / problem.number_of_variables, distribution_index=20),
    crossover=SBXCrossover(probability=1.0, distribution_index=20),
    termination_criterion=StoppingByEvaluations(max_evaluations=25000)
)

algorithm.run()

front = get_non_dominated_solutions(algorithm.get_result())

plot_front = Plot(title='Pareto front approximation', axis_labels=['x', 'y'])
plot_front.plot(front, label='MOCell-VRP', filename='MOCell-VRP', format='png')

In [43]:
from VRPTW.parser import SolomonFormatParser

problem = SolomonFormatParser("instances/C108.txt").get_problem()
problem

Instance: C108
Vehicle number: 25
Vehicle capacity: 200

In [14]:
C9?

Init signature: C9(*args, **kwds)
Docstring:     
 Class defining an C9 neighborhood of a solution belonging to a list of solutions which is
 structured as a bi-dimensional mesh. The neighbors are those solutions that are in 1-hop distance

Shape:
        * * *
        * o *
        * * *

Topology:
         north      = {-1,  0}
         south      = { 1 , 0}
         east       = { 0 , 1}
         west       = { 0 ,-1}
         north_east = {-1,  1}
         north_west = {-1, -1}
         south_east = { 1 , 1}
         south_west = { 1 ,-1}
 
File:           ~/anaconda3/lib/python3.8/site-packages/jmetal/util/neighborhood.py
Type:           ABCMeta
Subclasses:     
